Dictum provides data visualization capabilities by integrating into the great
[Altair](https://altair-viz.github.io/) library. Altair's declarative approach to dataviz
is very much in tune with Dictum's declarative approach to computing metrics.

Altair produces JSON chart "configs" that Dictum can understand and translate to actual
semantic queries. After retrieving the necessary data from your backend, Dictum will
return a new config with all the information that's necessary for correctly displaying
the chart.

If you're already familiar with Altair, here's the TLDR version of this section: you can
put a metric or a dimension anywhere you can put a field shorthand.

Metrics and dimensions "live" in the project, under `project.dimensions` and
`project.metrics` attributes.

In [11]:
from dictum import Project

project = Project.example("chinook")
m, d = project.metrics, project.dimensions
m.revenue

Let's make some charts! To create a chart, call the `project.chart()` instead of 
the usual `altair.Chart(data)`.

In [12]:
project.chart().mark_bar().encode(
    x=d.Year,
    y=m.revenue,
)

alt.Chart(...)

Dictum sets the default axis labels and formatting for you, as well as tooltips.
You can always override them in Altair.

Let's add different axis labels and some color.

In [13]:
from altair import X, Y, Color

(
    project.chart()
    .mark_bar()
    .encode(
        x=d.Year,
        y=Y(m.revenue, axis={"title": "Yearly Revenue", "format": "d"}),
        color=Color(d.Time.quarter),
    )
)

alt.Chart(...)

Metric and dimension transforms work the same way as with the query language and
Python API.

In [14]:
(
    project.chart()
    .mark_bar()
    .encode(
        x=d.Year,
        y=Y(m.revenue.percent(within=[d.Year])),
        color=Color(d.Time.quarter),
    )
)

alt.Chart(...)

If you need to use Dictum filters or limits, you have to call them right after `chart()`.

Let's see top 10 genres of music by revenue.

In [15]:
(
    project.chart()
    .where(d.music)
    .limit(m.revenue.top(10))
    .mark_bar()
    .encode(
        x=m.revenue,
        y=Y(d.genre, axis={"title": "Genre of Music"}, sort="-x"),
    )
)


alt.Chart(...)

You can use repeat to select multiple metrics.

In [16]:
import altair as alt


(
    project.chart()
    .where(d.music)
    .limit(m.revenue.top(10))
    .mark_bar()
    .encode(
        x=X(alt.repeat(), type="quantitative"),
        y=Y(d.genre, axis={"title": "Genre of Music"}, sort="-x"),
    )
    .properties(width=200)
    .repeat([m.revenue, m.unique_paying_customers])
)


alt.RepeatChart(...)